In [1]:
# Import libraries
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
from datetime import datetime, date
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno 

In [2]:
# Read dataset
df1 = pd.read_csv("docker_retail_masking (1).csv")
df2 = pd.read_csv("project_masking.csv")
df3 = pd.read_csv("so_retail_new (1).csv") 
df1

FileNotFoundError: [Errno 2] File b'docker_retail_masking (1).csv' does not exist: b'docker_retail_masking (1).csv'

In [ ]:
df2

In [ ]:
df3.rename(columns={'ID':'id'}, inplace=True)
df3

In [ ]:
# Docker Retail = Kolom Supir :
## Lower Case
df1["Supir"] = df1["Supir"].str.lower()

## Fill Null = -
df1["Supir"] = df1["Supir"].fillna("-")
df1[["Supir"]]

In [ ]:
### Get 'kode' from nomor_retail & docket_number_final
get = df1['nomor_retail'].str.split("/", n = 2, expand=True)
df1['kode_unit'] = get[1]

it = df1['docket_number_final'].str.split("/", n = 2, expand=True )
df1['kode_organisasi'] = it[1]

In [ ]:
### List Kode Divisi
kode_div = ['DV1', 'DV2', 'DV3', 'DOFF']
df3['kode_div'] = np.where(df3["kode_div"].str.contains('|'.join(kode_div)),df3['kode_div'], np.nan)
df3['kode_div'].fillna('-', inplace=True)

In [ ]:
### List Nama Divisi
df2['plant_nama'] = df2['plant_nama'].str.lower()
nama_div = ['divisi manufaktur', 'divisi precast', 'divisi property', 'divisi readymix']
df2['nama_div'] = np.where(df2["plant_nama"].str.contains('|'.join(nama_div)), df2['plant_nama'], np.nan)
df2['nama_div'].fillna('-', inplace=True)
df2['nama_div']

In [ ]:
### Plant_nama disesuaikan dengan objektif kata 'Plant'
plant_nama = ['plant bandara', 'plant bandung barat',
       'plant bandung timur', 'plant beam surabaya', 'plant bsd',
       'plant cakung', 'plant casablanca', 'plant cibitung',
       'plant cikarang', 'plant cikupa', 'plant cilacap', 'plant dadap',
       'plant dawuan', 'plant delta mas cikarang', 'plant depo lrt',
       'plant icon', 'plant kebon jeruk', 'plant kemayoran',
       'plant lenteng agung', 'plant mauk',
       'plant pile & slab jakarta xxx', 'plant precast cibitung',
       'plant precast cikarang', 'plant precast cilacap',
       'plant precast karawang', 'plant precast koala tanjung',
       'plant precast osowilangun', 'plant pulo gadung',
       'plant readymix cibubur', 'plant readymix cilacap',
       'plant readymix osowilangun', 'plant readymix precast cilacap',
       'plant readymix tambak sumur', 'plant readymix transpark bekasi',
       'plant readymix trembesi - kemayoran', 'plant sentul',
       'plant site mix kertajati cirebon',
       'plant site mix readymix palimanan', 'plant taman anggrek ii',
       'plant tanah abang']
df2['plant_nama'] = np.where(df2["plant_nama"].str.contains('|'.join(plant_nama)), df2['plant_nama'], np.nan)
df2['plant_nama'].fillna('-', inplace=True)

In [ ]:
# SO_Retail
## Mutu dan Spesifikasi lower case
df3['Spesifikasi'] = df3['Spesifikasi'].str.lower()
df3['Mutu'] = df3['Mutu'].str.lower()

## Buat kolom baru dari beberapa values kolom Spesifikasi
### Mutu
Moet = df3['Mutu'].str.split(n=1, expand=True)
df3['Mutu'] = Moet[0]

### Spesifikasi
df3['Spesifikasi_new'] = np.where(df3["Mutu"].str.contains('fa'),df3['Mutu'], 
                                  np.where(df3['Mutu'].str.contains('nfa'), df3['Mutu'],''))
### Spek_Tambahan
df3['Spesifikasi_tambahan'] = np.where(df3["Spesifikasi"].str.contains('spesial'),df3['Spesifikasi'], 
                                  np.where(df3['Spesifikasi'].str.contains('fast'), df3['Spesifikasi'],
                                          np.where(df3['Spesifikasi'].str.contains('track'), df3['Spesifikasi'],
                                                  np.where(df3['Spesifikasi'].str.contains('fastrack'), df3['Spesifikasi'],
                                                          np.where(df3['Spesifikasi'].str.contains('ctb'), df3['Spesifikasi'],
                                                                  np.where(df3['Spesifikasi'].str.contains('porous'), df3['Spesifikasi'],''))))))

In [ ]:
### Spesifikasi_new split FA dan NFA
new = df3['Spesifikasi_new'].str.split(' ', n=1, expand=True)
df3['Spesifikasi_new'] = new[1]
df3['Spesifikasi_new'] = df3['Spesifikasi_new'].str.upper()
df3['Spesifikasi_new'] = df3['Spesifikasi_new'].fillna('-')

In [ ]:
### Spesifikasi_tambahan
#### Replace beberapa values sesuai dengan dictionary
##### 'spesial 1 hari'
df3['Spesifikasi_tambahan'].replace(['fast track 1 hari'], 
                                    'spesial 1 hari', inplace=True)

##### 'spesial 2 hari'
df3['Spesifikasi_tambahan'].replace(['k-600 nfa spesial kering 2 hari'], 
                                    'spesial 2 hari', inplace=True)

##### 'spesial 3 hari'
df3['Spesifikasi_tambahan'].replace(['k-350 nfa (fast track 3 hari)', 'fast track 3 hari',
                                     'fs-45 nfa spesial 3 hari', 'k-300 nfa fastrack 3 hari',
                                     'fast track 3 hari ', 'k-400 nfa fast track 3 hari',
                                     'k-350 nfa spesial 3 hari', 'class p-1 nfa spesial 3hari',
                                     'k 300 nfa spesial 3 hari', 'class p-1 nfa spesial 3 hari',
                                     'k-300 fa fast track 3 hari', ' 3 hari track fasttsafk-400 nfa',
                                     'fastrack 3 hari ', 'fs-45 fa fastrack 3 hari',
                                     'fc-45 nfa fastrack 3 hari', 'spesial 3 hari 100%',
                                     'fast trak 3 hari', 'k-500 nfa spesial 3 hari',
                                     'fs-45 nfa (fastrack 3 hari)', 'fs-45 nfa (fastrack 3 hari )', 
                                     'spesial 3 hari fs-45 nfa', 
                                     'fs-45 nfa  (fastrack 3 hari )', 'spesial 3 harifs-45 nfa',
                                     'fs-45 nfa spesial 3 hari ', 'fs-45 nfa (fast track 3 hari )',
                                     '\tfastrack 3 hari', 'fs-45 nfa  fastrack 3 hari',
                                     ' fastrack 3 hari', 'spesial 3 hari ',
                                     '(fastrack 3 hari)', ' (fastrack 3 hari)', 
                                     'fastrack  3 hari', 'fastrack 3 hari\t', 
                                     'fastrack 3 hari','fc-40 nfa spesial 3 hari',
                                     'k-350 nfa fast track 3 hari scr '], 
                                    'spesial 3 hari', inplace=True)

##### 'spesial 7 hari'
df3['Spesifikasi_tambahan'].replace(['fast track 7 hari',
                                     'k-350 nfa spesial 7 hari',
                                     'spesial 7 hari ',
                                     'fc-30 nfa spesial 7 hari'], 
                                    'spesial 7 hari', inplace=True)

##### 'spesial 10 hari'
df3['Spesifikasi_tambahan'].replace(['k-225 nfa spesial 10 hari'], 
                                    'spesial 10 hari', inplace=True)

##### 'spesial 14 hari'
df3['Spesifikasi_tambahan'].replace(['k-225 nfa fast track 14 hari',
                                      'k-350 nfa spesial 14 hari',
                                     'fast track 14 hari',
                                     'k-300 nfa spesial 14 hari',
                                     'k-300 fa spesial 14 hari'], 
                                    'spesial 14 hari', inplace=True)
##### 'spesial 28 hari'
df3['Spesifikasi_tambahan'].replace(['fs-45 nfa (spesial 28 hari)', 
                                     'fs-45 nfa spesial 28 hari'], 
                                    'spesial 28 hari', inplace=True)

##### 'ctb'
df3['Spesifikasi_tambahan'].replace(['k-225 nfa spesial 10 hari', 'ctb k125',
                                    'ctb k125', 'ctb-c-0 fa k125',
                                    'k-150 nfa ctb', 'k-300 fa ctb', 
                                    'ctb-c-0 fa k 150','ctb-c-0 fa k 125'], 
                                    'ctb', inplace=True)

##### 'spasi'
df3['Spesifikasi_tambahan'].replace([''], 
                                    '-', inplace=True)

##### fillna('-')
df3['Spesifikasi_tambahan'].fillna('-', inplace=True)

In [ ]:
### is_web == retail_method
df3['is_website'] = np.where(df3["retail_method"].str.contains('manual'),'false','true') 
df3[['retail_method','is_website']]    

In [ ]:
## Alamat_proyek
df3['Alamat_proyek'] = df3['street'] +'- '+ df3['street2']
df3

In [ ]:
### change start_time type from object to datetime/timestamp
df1['start_time'] = pd.to_datetime(df1.start_time)

In [ ]:
### Fill null in realisasi_volume_terima with mean/average
df1['realisasi_volume_terima'].fillna(df1['realisasi_volume_terima'].mean(), inplace=True)

In [ ]:
## inner join df1, df2 & df3
result = pd.concat([df1, df2, df3], keys=['df1', 'df2', 'df3'], axis=1, join='inner')
result

In [ ]:
### Drop some columns
result.isnull().sum()

In [ ]:
# kode_unit = ['U1-2', 'UR', 'U2-1', 'U1-1', 'U2-3', 
#              'U1-4', 'U1-3', 'U1-5','U2-2', 'U2-4', 
#              'U3-1', 'UT', 'False', 'RMC']

# kode_org = ['CSA', 'CBR', 'DWN', 'DMS', 'KJI', 'CKG', 'BSD', 'STL', 'TMA',
#        'DDP', 'PST', 'LTA', 'TSR', 'BTR', 'CKP', 'TNA', 'BBT', 'BKTR',
#        'GLE', 'MAK', 'CBT', 'CLP', 'KBJ', 'PLMN', 'MON', 'OW',
#        'TNA2', 'KLS', 'LTAO', 'KPO']

#### Kode_Unit
# df3['kode_unit'] = np.where(df3["kode_unit"].str.contains('|'.join(kode_unit)), df3['kode_unit'], 
#                             np.where(df3["kode_unit"].str.contains('|'.join(kode_org)), np.nan, np.nan))
# df3['kode_unit'].fillna('-')

# #### Kode_Organisasi
# df3['kode_organisasi'] = np.where(df3["kode_organisasi"].str.contains('|'.join(kode_org)), df3['kode_organisasi'], 
#                             np.where(df3["kode_organisasi"].str.contains('|'.join(kode_div)), np.nan, 
#                                      np.where(df3["kode_organisasi"].str.contains('|'.join(kode_unit)), np.nan, np.nan)))
# df3['kode_organisasi'].fillna('-')